In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
## Imports and environment variables 
import os
import torch

from travis_attack.utils import set_seed, set_session_options, setup_logging, resume_wandb_run, display_all
from travis_attack.config import Config
from travis_attack.models import prepare_models, get_optimizer
from travis_attack.data import ProcessedDataset
from travis_attack.trainer import Trainer
from travis_attack.insights import (postprocess_df, create_and_log_wandb_postrun_plots, 
                                    get_training_dfs, get_interesting_idx, print_stats, print_interesting_text_stats)

In [ ]:
import logging 
logger = logging.getLogger("run")

In [ ]:
cfg = Config()
if cfg.dataset_name == "rotten_tomatoes": cfg = cfg.small_ds()
set_seed(cfg.seed)
set_session_options()
setup_logging(cfg, disable_other_loggers=False)
vm_tokenizer, vm_model, pp_tokenizer, pp_model, sts_model, cfg = prepare_models(cfg)
optimizer = get_optimizer(cfg, pp_model)
ds = ProcessedDataset(cfg, vm_tokenizer, vm_model, pp_tokenizer, sts_model, load_processed_from_file=True)

In [ ]:
# cfg.wandb['mode'] = 'disabled'
# trainer = Trainer(cfg, vm_tokenizer, vm_model, pp_tokenizer, pp_model, sts_model, optimizer,
#                   ds, initial_eval=False)
# trainer.train()

In [ ]:
## Test just the analysis bit
cfg.run_id = '61zaljb2'
cfg.run_name = "hearty-cherry-289"
cfg.path_run = f"{cfg.path_checkpoints}{cfg.run_name}/"
run = resume_wandb_run(cfg)


In [ ]:
df_d = get_training_dfs(cfg)
num_proc = 4
for k, df in df_d.items(): df_d[k] = postprocess_df(df, filter_idx=None, num_proc=num_proc)

In [ ]:
#create_and_log_wandb_postrun_plots(df_d)

In [ ]:
#get_run_information_markdown(cfg)

In [ ]:
idx_d = get_interesting_idx(df_d['training_step'], n=5)

In [ ]:
print_interesting_text_stats(df_d['training_step'], n=5)